In [59]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [60]:
URL()

URL('')

In [61]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [ ]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [ ]:
smartphones_json = to_json(smartphone_list)

In [ ]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *

In [2]:
df_brut = pl.read_json("data/smartphones.json")

In [3]:
df_brut.select(pl.all().is_null().sum())

model,sensor_resolution,cpu,upgrade_storage,sensor,sim_type,screen_size,repairability_index,thickness,das_limbs,brand,reviews,made_in,network,battery,fast_charging,color,charging_type,screen_type,price,warranty,net_weight,screen_resolution,ram,sim_number,height,stars,width,bluetooth,storage,screen_tech,os,das_chest,das_head,usb_type_c,refresh_rate,cpu_details
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,1,5,1,81,0,0,16,3,13,0,0,48,0,24,2,1,17,4,0,92,18,0,29,5,3,0,3,0,0,2,2,13,13,86,337,210


### Pipeline

Sur `535` extractions, il y a `436` téléphones valides avec les caractéristiques. Pas mal ?

Pour obtenir la *PPI* (Pixels per Inch) on fait : 

$$PPI = \dfrac{\text{diagonal\_pixels}}{\text{screen\_size} (inch)}$$

In [180]:
df = NordStream(df_brut)

In [181]:
df = df.with_columns(
    pl.col("sensor_resolution")
    .str.extract_all(r"(\d+)")
    .list.to_struct(fields=["cam_1", "cam_2", "cam_3"])
).unnest("sensor_resolution").with_columns(
    pl.col("cam_1", "cam_2", "cam_3").cast(pl.Float64).fill_null(strategy="zero")
).with_columns(
    pl.col("cam_1").add(pl.col("cam_2").add(pl.col("cam_3"))).alias("mpx_backward_cam")
)

In [50]:
#df.select(cs.by_dtype(pl.NUMERIC_DTYPES)).corr()

# Matrice de corrélation

In [182]:
import statsmodels.formula.api as smf
import statsmodels.formula as sm
from stargazer.stargazer import Stargazer
import numpy as np
import random
from dataclasses import dataclass

In [183]:
df = df.drop_nulls(subset=cs.contains({"made_in", "repairability_index", "width"}))

In [185]:
df = df.with_columns(random_col = pl.lit(0))

In [186]:
df = df.with_columns(pl.col("random_col").map_elements(lambda x: random.randint(-1000, 1000)))

In [10]:
@dataclass
class Formula:
    y: str
    X: str

    def __repr__(self):
        return f"{self.y} ~ {self.X}"

In [187]:
X2 = "screen_tech"

In [218]:
y = "price"
X = "storage + induction + ram + screen_type + network + diagonal_pixels + screen_size + brand + made_in + das_limbs"

In [219]:
f = Formula(y, X)

Variables qui n'apportent rien dans la loglin-reg : 

- color
- net_weight
- model
- das_chest
- das_head

In [220]:
def execute_ols(df: pl.DataFrame, f: Formula):
    ols_model = smf.ols(
        formula=str(f),
        data=df,
    )
    res = ols_model.fit()
    #coefs = dict(res.params)
    return res

In [221]:
res = execute_ols(df, f)

In [222]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     126.4
Date:                Wed, 01 Nov 2023   Prob (F-statistic):          2.29e-179
Time:                        12:07:05   Log-Likelihood:                -2735.6
No. Observations:                 428   AIC:                             5529.
Df Residuals:                     399   BIC:                             5647.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [223]:
def feature_importance(df: pl.DataFrame, y:str):
    column_name = list()
    r2_list = list()
    for column in df.columns:
        ols_model = smf.ols(
        formula=f"{y}~{column}",
        data=df)
        res = ols_model.fit()
        column_name.append(column)
        r2_list.append(res.rsquared)

    feature_imp_df = pl.DataFrame({'Column': column_name, 'R-squared': r2_list}).sort("R-squared", descending=True)
    return feature_imp_df

In [227]:
df.group_by("screen_tech").count()

screen_tech,count
str,u32
"""OLED QHD+""",3
"""Retina [LCD]""",5
"""AMOLED (rigide…",38
"""pOLED HDR10""",13
"""LCD""",9
"""Non communiqué…",69
"""Actua ou OLED …",6
"""Super Retina X…",41
"""AMOLED""",3


In [224]:
df_features = feature_importance(df, "price")

In [199]:
import plotly.express as px

In [225]:
fig = px.bar(df_features, x='Column', y='R-squared', title='Feature Importance (R-squared)')
fig.update_xaxes(title_text='Feature')
fig.update_yaxes(title_text='R-squared')
fig.show()

In [ ]:
def predict_phone_price(df: pl.DataFrame, f: Formula, storage: float, ram: float):
    res = execute_ols(df, f)
    params = dict(res.params)
    prediction = params["Intercept"] + params["storage"]*storage + params["ram"]*ram
    return prediction

In [ ]:
predict_phone_price(df, f, 32, 1)

143.2669566783436

In [ ]:
ols_model = smf.ols(
    formula=f"{y}~{X2}",
    data=df,
)

In [ ]:
res = ols_model.fit()

In [ ]:
dict(res.params)

{'Intercept': 1071.5195276459249,
 'storage': 1.1958931708843838,
 'ram': 53.85417103504183,
 'battery': -0.23112324797420558}

In [ ]:
real_price = df.select("price").to_numpy()

In [ ]:
price_predict = res.predict()

In [ ]:
# res2 = res.get_robustcov_results(cov_type='HC1')
# robuste std errors

In [66]:
Stargazer([res])

- fuzzy match à mettre en place ici (pour la technologie de l'écran) ?
- ajouter une feature complètement random (pour la partie Machine Learning) puis regarder l'importance des variables : si les variables sont moins importantes que la random, alors on les dégage

In [ ]:
df.group_by("brand").count().sort("count")

brand,count
str,u32
"""FAIRPHONE""",3
"""ONEPLUS""",7
"""NOTHING""",7
"""VIVO""",9
"""ASUS""",12
"""REALME""",14
"""HONOR""",14
"""SONY""",15
"""OPPO""",18


In [ ]:
df.select("price", "storage").group_by("storage").mean().sort("price")

storage,price
i64,f64
32,116.647407
64,213.197037
128,496.864041
256,872.482936
512,1214.744324
1000,1752.29375


In [ ]:
df.select("price", "ram").group_by("ram").mean().sort("price")

ram,price
str,f64
"""2 Go""",110.109048
"""3 Go""",158.240588
"""4 Go""",413.402542
"""6 Go""",462.522203
"""8 Go""",686.67741
"""16 Go""",1106.142857
"""12 Go""",1144.023733


In [ ]:
df.select("price", "screen_type").group_by("screen_type").mean()

screen_type,price
str,f64
"""Plat""",575.627179
"""Pliable""",1436.472778
"""Bords incurvés…",654.381111
"""Borderless""",386.24338


In [ ]:
import plotly.express as px

fig = px.histogram(df, x="diagonal_pixels", histnorm="probability")
fig.update_layout(
    title="Distribution des diagonal_pixels",
    xaxis_title="diagonal_pixels",
    yaxis_title="Fréquence",
)
fig.show()

**TODO** `cleaning` :

- [x] **network**
- [x] **color**
- [x] **price**
- [x] **screen_type**
- [ ] *cpu*

In [ ]:
# df.select("cpu").unique().to_numpy()
# pour le cpu on va préférer retravailler pour avoir la fréquence en GHz plutot

Vérification des valeurs nulles sur les différentes colonnes.

In [ ]:
df.select(pl.all().is_null().sum())

screen_tech,upgrade_storage,height,cpu,reviews,storage,color,fast_charging,sensor,model,sensor_resolution,ram,charging_type,repairability_index,das_chest,screen_size,network,net_weight,stars,das_limbs,battery,made_in,price,width,sim_type,thickness,das_head,screen_type,usb_type_c,brand,diagonal_pixels
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,1,1,0,0,1,0,75,0,0,0,3,0,0,0,0,14,0,0,0,7,0,1,0,1,0,0,0,0,0


In [ ]:
df.select(pl.col("model").filter(pl.col("net_weight").is_null())).unique().to_numpy()

array([['Samsung Galaxy S10'],
       ['Google Pixel 8'],
       ['Google Pixel 8 Pro'],
       ['Sony Xperia 1 V']], dtype=object)

Idée => Avec cette liste unique des téléphones, aller chercher leur poids (manquant) avec une `request` sur google et récupérer le résultat